In [ ]:
!apt-get update
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,586 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://security.

In [ ]:
!pip install colab-env -qU
import colab_env

  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive
Creating vars.env in your Google Drive!


In [ ]:
!pip install python-dotenv

import os

import dotenv
from google.colab import drive

drive.mount('/content/drive')

dotenv.load_dotenv('/content/drive/MyDrive/cupy.env')

os.environ.get('CUDA_ACCELERATOR')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'cutensor'

In [ ]:
print(os.environ.get('CUDA_ACCELERATOR'))

cutensor


In [1]:
pip install cupy-cuda12x cutensor-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 6.4 MB/s eta 0:00:00


In [14]:
import numpy as np
import cupy as cp
import cutensor as cutensor

cp.cuda.Stream.null.synchronize()

In [3]:
arr = cp.ones((1000,500,500))

#makes us wait for gpu to finish before returning
cp.cuda.Stream.null.synchronize()

In [4]:
A = cp.array([[[1,2,3],
               [3,4,5],
               [6,7,8]],

              [[9,10,11],
               [12,13,14],
               [15,16,17]]],)

B = cp.random.rand(3,3,3)


print("A: ", A)
print("B: ", B)
print(A.shape)
print(B.shape)


A:  [[[ 1  2  3]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]]
B:  [[[0.89037304 0.51555246 0.74820862]
  [0.85194354 0.51516188 0.93383825]
  [0.40876618 0.98815755 0.14175375]]

 [[0.41463821 0.25768896 0.26773643]
  [0.90078436 0.60219592 0.7972908 ]
  [0.82448646 0.2639169  0.23354816]]

 [[0.31462561 0.01686404 0.67710199]
  [0.46265903 0.6170781  0.86759747]
  [0.32030822 0.70509691 0.65776888]]]
(2, 3, 3)
(3, 3, 3)


In [ ]:
result = cp.einsum('ijk,jkl->il', A, B)
print("Result with cp.einsum: ", result)

# ijk the dimensions of A, jkl the dimensions of B, il the dimensions of the einstein sum
# contraction happens over the j and k dimensions
cp.cuda.Stream.null.synchronize()

Result with cp.einsum:  [[17.63499399 16.34839075 23.61003662]
 [53.88210597 54.03773114 65.03892462]]


In [ ]:
# using cutensor

#need descriptors to describe shape, data type and memory of tensor
desc_A = cutensor.create_tensor_descriptor(A)

desc_B = cutensor.create_tensor_descriptor(B)

# Empty output tensor of the shape of the einstein sum of dimensions il
output = cp.empty((2,3), dType = A.dtype)

desc_output = cutensor.create_tensor_descriptor(output)


AttributeError: module 'cutensor' has no attribute 'create_tensor_descriptor'

In [ ]:
#create description of contraction of the tensors
contract_desc = cutensor.einsum_expr(('i','j','k'),('j','k','l'),('i','l'))
#('i','j','k') dimensions of A,
#('j','k','l') dimensions of B,
#('i','l') dimensions of output

#scale factor for contraction operation
alpha = 1.0
#scale factor for output
beta = 0.0

#contraction plan
contraction_plan = cutensor.create_contraction_plan(contract_desc, desc_A, desc_B, desc_output)

In [ ]:
#Perform contraction using contraction plan
cutensor.contraction(contraction_plan, alpha, A, desc_A, B, desc_B, beta, output, desc_output)

cp.cuda.Stream.null.synchronise()
print("Result with cutensor: ", output)

In [5]:

#Task 1 function/abstract contraction passing the tensor, matrix, and dimension

def contract(A, X, dim):
    # Takes tensor A of unknown number of dimensions
    # Takes matrix h with 2 dimensions Kk
    # Takes int dim as the dimension to contract over
    if (type(dim) != int and type(dim) == str) :
      raise ValueError(f"Dim must be an int not a string")

    # Get the dimensions of the tensor and matrix
    A_shape = cp.array(A.shape)
    X_shape = cp.array(X.shape)

    # Check that the size of the dimension matches
    if A_shape[dim] != X_shape[1]:
      raise ValueError(f"Dimension mismatch: A has size {A_shape[dim]} along dimension {dim}, "f"but X has size {X_shape[1]}")

    # Compress the dimensions not involved in contraction

    reshape_left = cp.prod(A_shape[:dim]).item()
    contract_dim_size = A_shape[dim].item()
    reshape_right = cp.prod(A_shape[dim+1:]).item()

    #print(f"Reshape Left: {reshape_left} (type: {type(reshape_left)})")
    #print(f"Contract Dim Size: {contract_dim_size} (type: {type(contract_dim_size)})")
    #print(f"Reshape Right: {reshape_right} (type: {type(reshape_right)})")


    As = A.reshape(reshape_left, contract_dim_size, reshape_right)

    result = cp.einsum("jkl, Kk->jKl", As, X)
    #print(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())
    result = result.reshape(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())

    return result

    #As = A.reshape( np.prod(A.shape[:2]), A.shape[2], np.prod(A.shape[3:]))
    #Ahs = np.einsum("Kk,ikl->iKl",h,As)
    #np.allclose(Ahs.ravel(),Ah.ravel())
#
#
    #In [17]: A = np.random.rand(3,2,4,6,1)

    #In [18]: h = np.random.rand(4,4)

    #In [19]: Ah = np.einsum("Kk,ijklm->ijKlm",h,A)

    #In [20]: As = A.reshape( np.prod(A.shape[:2]), A.shape[2],
    #np.prod(A.shape[3:]))

    #In [21]: Ahs = np.einsum("Kk,ikl->iKl",h,As)

    #In [22]: np.allclose(Ahs.ravel(),Ah.ravel())
    #In [23]: np.allclose(Ahs.reshape(Ah.shape),Ah)
#
    #A = A.reshape(A.prod(A.shape[:]))



In [ ]:
A = np.random.rand(2,3,2,4,7,8)
h = np.random.rand(4,7)
Ah = np.einsum("Kk,hijlkm->hijlKm",h,A)

Ahs = contract(A, h, 4)

print(cp.allclose(Ahs.ravel(),Ah.ravel()))
print(len(Ahs.ravel()))
print(len(Ah.ravel()))
print(cp.allclose(Ahs.reshape(Ah.shape),Ah))
print(cp.allclose(Ahs,Ah))

True
1536
1536
True
True


In [11]:
A = np.random.rand(2,7,8)
h = np.random.rand(4,7)
Ah = np.einsum("Kk,hkm->hKm",h,A)

Ahs = contract(A, h, 1)
print(Ahs, " hs")
print(Ah)

[[[1.89546492 0.99920597 1.65559785 1.28241343 1.63012005 1.03819686
   1.01573991 2.23023089]
  [2.41601511 1.47243827 2.03740346 1.81542522 2.65966053 1.77995681
   1.52068415 2.71345123]
  [1.6024132  0.83898067 1.21026019 0.88208145 1.19474929 0.81318585
   1.01715205 1.7361451 ]
  [2.2648725  1.24998627 1.33162126 1.07031665 2.08061963 1.27806666
   1.73700466 2.19160274]]

 [[1.92945463 1.67204674 2.73611767 1.27540545 1.26966615 1.47690926
   1.38953016 2.24160723]
  [1.85853126 2.3912412  3.37542082 2.03904117 1.54999492 2.28167553
   1.54723588 2.79278733]
  [1.51175611 1.46333623 1.88953545 0.89352247 1.01780147 0.9705407
   0.9917775  1.56028492]
  [1.75181629 2.39890555 2.19394754 1.24637391 1.42086027 1.40715606
   1.17433515 1.97622764]]]  hs
[[[1.89546492 0.99920597 1.65559785 1.28241343 1.63012005 1.03819686
   1.01573991 2.23023089]
  [2.41601511 1.47243827 2.03740346 1.81542522 2.65966053 1.77995681
   1.52068415 2.71345123]
  [1.6024132  0.83898067 1.21026019 0.88208

In [ ]:
import numpy as np
import cupy as cp
import time
from cupy import cutensor as cutensor

cp.cuda.Stream.null.synchronize()


#arr = cp.ones((1000,500,500))

#makes us wait for gpu to finish before returning
cp.cuda.Stream.null.synchronize()


#A = cp.array([[[1,2,3],
#               [3,4,5],
#               [6,7,8]],
#
#              [[9,10,11],
#               [12,13,14],
#               [15,16,17]]],)

#B = cp.random.rand(3,3,3)


#print("A: ", A)
#print("B: ", B)
#print(A.shape)
#print(B.shape)


def contract(A, X, dim):
    # Takes tensor A of unknown number of dimensions
    # Takes matrix h with 2 dimensions Kk
    # Takes int dim as the dimension to contract over
    if (type(dim) != int and type(dim) == str) :
      raise ValueError(f"Dim must be an int not a string")

    # Get the dimensions of the tensor and matrix
    A_shape = cp.array(A.shape)
    X_shape = cp.array(X.shape)

    # Check that the size of the dimension matches
    if A_shape[dim] != X_shape[1]:
      raise ValueError(f"Dimension mismatch: A has size {A_shape[dim]} along dimension {dim}, "f"but X has size {X_shape[1]}")

    # Compress the dimensions not involved in contraction

    reshape_left = cp.prod(A_shape[:dim]).item()
    contract_dim_size = A_shape[dim].item()
    reshape_right = cp.prod(A_shape[dim+1:]).item()

    #print(f"Reshape Left: {reshape_left} (type: {type(reshape_left)})")
    #print(f"Contract Dim Size: {contract_dim_size} (type: {type(contract_dim_size)})")
    #print(f"Reshape Right: {reshape_right} (type: {type(reshape_right)})")

    As = A.reshape(reshape_left, contract_dim_size, reshape_right)

    result = cp.einsum("jkl, Kk->jKl", As, X)
    #print(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())
    result = result.reshape(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())

    return result


def contract_with_plan(A, X, dim):
    if (type(dim) != int and type(dim) == str) :
        raise ValueError(f"Dim must be an int not a string")

    A_shape = cp.array(A.shape)
    X_shape = cp.array(X.shape)

    if A_shape[dim] != X_shape[1]:
      raise ValueError(f"Dimension mismatch: A has size {A_shape[dim]} along dimension {dim}, "f"but X has size {X_shape[1]}")


    A_dims = len(A_shape)
    X_dims = len(X_shape)

    if X_dims > 2:
        raise ValueError(f"Contractiong matrix has {X_dims} dimensions when it should only have 2")

    A_mode = tuple(map(chr, range(97, 97 + A_dims)))
    X_mode = tuple(chr(97 + A_dims), A_mode[dim])

    result_mode = tuple(A_mode[:dim] + X_mode[0] + A_mode[dim+1:])
    result_shape = A_shape[:dim] + X_shape[0] + A_shape[dim+1:]
    result_empty = cp.empty(result_shape)

    alpha = 1.0
    beta = 0.0
    result = cutensor.contraction(alpha, A, A_mode, X, X_mode, beta, result_empty, result_mode)

    return result


alpha = 1.0
beta = 0.0
mode_a = ('a', 'b', 'c')
mode_b = ('d', 'b')
mode_c = ('d', 'e')
mode_ab = ('a','d', 'c')
mode_abc = ('a', 'e')
a = cp.random.random((3, 4, 5))
b = cp.random.random((6, 4))
c = cp.random.random((6, 7))
ab = cp.empty((3, 6, 5))
abc = cp.empty((3, 7))

result_AB = cutensor.contraction(alpha, a, mode_a, b, mode_b, beta, ab, mode_ab)

result_AB_func = contract_with_plan(a, b, 1)

cp.cuda.Stream.null.synchronize()

print(cp.allclose(result_AB, result_AB_func))

#cutensor.contraction(alpha, ab, mode_ab, c, mode_c, beta, abc, mode_abc)
#
#cp.cuda.Stream.null.synchronize()
#
#
#size = A.data.nybytes

#print(result_AB)
#print(contract(a, b, 1))

#twrite = []
#for i in range(10):
#    a = cp.random.random((3, 4, 5))
#    b = cp.random.random((6, 4))
#    ab = cp.empty(3,6,5)
#    t1 = time.time()
#    result_AB_func = contract_with_plan(a, b, 1)
#    cp.cuda.Stream.null.synchronize()
#    t1 = time.time() - t1
#    twrite.append(t1)
#
#times = np.array(twrite)

#print(f"\tWRITE  mean= {times.mean():12.5f} max= {times.max():12.5f} min= {times.min():12.5f} std= {times.std():12.5f}",flush=True)
